In [ ]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:37841")
client

In [ ]:
conf = {
    'bootstrap.servers': 'kafka-cluster-kafka-bootstrap:9092',
    'group.id': 'daks-consumers',
    'auto.offset.reset': 'earliest',
    'enable.auto.commit': 'false'
}
topic = 'topic4-real'

In [ ]:
import pandas as pd
import dask.dataframe as dd

from dask_kafka import KafkaReader

kr = KafkaReader(client, topic, conf)

# read batch from kafka with dask
batch_fut = kr.read_batch()

# check if it is not empty
if len(batch_fut)>0:
    # convert results in pandas dataframes (one per worker)
    fut_res = [client.submit(pd.DataFrame, b_fut) for b_fut in batch_fut]

    # create dask dataframe
    df = dd.from_delayed(fut_res).compute()

    df.head()